In [ ]:
# Download and install
curl https://clickhouse.com/ | sh

# Verify
clickhouse local --query "SELECT version()"

# Interactive mode
clickhouse local

In [ ]:
services:
  clickhouse:
    image: clickhouse/clickhouse-server:latest
    container_name: clickhouse
    ports:
      - "8123:8123"
      - "9000:9000"
    environment:
      CLICKHOUSE_USER: ashwin
      CLICKHOUSE_PASSWORD: mysecret
      CLICKHOUSE_DEFAULT_ACCESS_MANAGEMENT: 1
    volumes:
      - clickhouse_data:/var/lib/clickhouse

volumes:
  clickhouse_data:


In [ ]:
After install verify going in the container:

docker exec -it clickhouse clickhouse-client -u ashwin --password


In [ ]:
connect to port 8123 in dbweaver

In [ ]:
CREATE DATABASE testdb;

USE testdb;

CREATE TABLE employees
(
    id UInt32,
    name String,
    salary UInt32,
    created_at DateTime
)
ENGINE = MergeTree()
ORDER BY id;

INSERT INTO employees VALUES
(1,'Ashwin',50000, now()),
(2,'Ram',60000, now()),
(3,'Sita',55000, now());


In [ ]:
CREATE TABLE events (
    timestamp DateTime,
    user_id UInt64,
    event_type String
) ENGINE = MergeTree()
ORDER BY (timestamp, user_id);

INSERT INTO events VALUES
    ('2024-01-15 10:00:00', 1, 'click'),
    ('2024-01-15 10:05:00', 2, 'view'),
    ('2024-01-15 10:10:00', 1, 'click'),
    ('2024-01-15 10:15:00', 3, 'view');

SELECT event_type, count() as count
FROM events
GROUP BY event_type;

Generating Test Data

In [ ]:
-- Generate 1 million rows
INSERT INTO events
SELECT
    now() - (number * 60) as timestamp,
    number % 10000 as user_id,
    ['click', 'view', 'purchase'][number % 3 + 1] as event_type
FROM numbers(1_000_000);

-- Generate 1 billion rows (for performance testing)
INSERT INTO events
SELECT
    toDateTime('2024-01-01 00:00:00') + (number * 60) as timestamp,
    number % 1000000 as user_id,
    ['click', 'view', 'purchase'][number % 3 + 1] as event_type
FROM numbers(1_000_000_000);

queries are fast Because ClickHouse uses sparse primary index marks to jump directly to relevant ranges instead of scanning entire table.


In [ ]:
-- result of this query elapsed: 0.005s
Read: 57,344 rows (229.38 KB)

SELECT *
FROM events
WHERE timestamp >= '2024-03-01'
  AND timestamp < '2024-03-02';

Lets use Partitionby for this

In [ ]:
CREATE TABLE events (
    timestamp DateTime,
    user_id UInt64,
    event_type LowCardinality(String)
)
ENGINE = MergeTree()
PARTITION BY toYYYYMM(timestamp)
ORDER BY (timestamp, user_id);


1️⃣ Partitioning by Month

Now data is split physically like:
202401
202402
202403
...

In [ ]:
we have event_type as 
click
view
purchase

LowCardinality(String)
Reduces storage and speeds up GROUP BY.

Compression improves significantly at 1B rows.